In [ ]:
import pandas as pd
import tensorflow as tf
import keras
from tensorflow.keras import preprocessing
from keras.models import Model, load_model
from keras.layers import Input, Embedding, Dense, Dropout, Conv1D, GlobalMaxPool1D, concatenate
from tqdm import tqdm
import numpy as np
from gensim.models.word2vec import Word2Vec
from keras.initializers import Constant
from keras.callbacks import EarlyStopping
from keras.callbacks import ModelCheckpoint
from keras import Sequential
from keras.models import Sequential
from keras.layers import Bidirectional, LSTM, Dense, TimeDistributed

In [ ]:
#konlpy 설치
%%bash
apt-get update
apt-get install g++ openjdk-8-jdk python-dev python3-dev
pip3 install JPype1
pip3 install konlpy

Hit:1 http://archive.ubuntu.com/ubuntu bionic InRelease
Ign:2 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Get:3 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Hit:4 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Hit:5 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Get:6 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease [3,626 B]
Get:7 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Get:8 http://archive.ubuntu.com/ubuntu bionic-backports InRelease [83.3 kB]
Hit:9 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease
Hit:10 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelease
Hit:12 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu bionic InRelease
Hit:13 http://ppa.launchpad.net/graphics-drivers/ppa/ubuntu bionic InRelease
Fetched 264 kB in 2s (108 kB/s)
Reading p

In [ ]:
#경로 설정
%env JAVA_HOME "/usr/lib/jvm/java-8-openjdk-amd64"

env: JAVA_HOME="/usr/lib/jvm/java-8-openjdk-amd64"


In [ ]:
from konlpy.tag import Komoran

In [ ]:
#하이퍼 파라미터 조정을 위한 keras tuner 설치
!pip install -q -U keras-tuner
import kerastuner as kt

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
#데이터에서 라벨값(정답값) 불러오기
train_file='/content/drive/MyDrive/ContestDataAll11.csv'
data=pd.read_csv(train_file, encoding='cp949')
labels=data['label'].tolist()
len(labels)

81539

In [ ]:
#데이터에서 라벨값(정답값) 불러오기
train_file='/content/drive/MyDrive/ContestDataAll11.csv'
data=pd.read_csv(train_file, encoding='cp949')
labels=data['label'].tolist()
type(labels[0])

int

In [ ]:
#전처리한 데이터 불러오기
tokenized_data = pd.read_csv("/content/drive/MyDrive/ContestDataAll11_tokenize_final.csv", encoding = 'euc-kr', low_memory = False)
tokenized_data = tokenized_data.iloc[0:,1:]
arr = np.array(tokenized_data)
li = arr.tolist()

In [ ]:
tokenized_data

,0,1,2,3,4,5,6,7,8,9,10,11,12,13
0,추리,그러,꼭,1,st,시간,말,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2000,1,시각,정보,깨우,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,일,반,인,창원,저,를,떼,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,피자,로,1,번,띠,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,1,반,아바,ㄴ,떼,자로,나타내,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
81534,3,D,정보,모니터,끄,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
81535,3,D,정보,모니터,꺼,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
81536,3,D,정보,모니터,닫,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
81537,3,D,정보,모니터,종료,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [ ]:
#리스트내에서 nan값 제거하기
tokenize_list =[]
for i in range(len(li)):
    tmp=[]
    for j in range(len(li[0])):
        if li[i][j] is not np.nan:
            tmp.append(li[i][j])
    tokenize_list.append(tmp)

In [ ]:
len(tokenize_list)

81539

In [ ]:
tokenize_list[80642]

['3', '차원', '정보', '종료']

In [ ]:
#토크나이징한 단어들을 임베딩
tokenizer=preprocessing.text.Tokenizer(oov_token='OOV')
tokenizer.fit_on_texts(tokenize_list)
sequences=tokenizer.texts_to_sequences(tokenize_list)
word_index=tokenizer.word_index
len(word_index)

617

In [ ]:
sequences[80642]

[4, 8, 7, 67]

In [ ]:
word_index

{'OOV': 1,
 '번': 2,
 '시각': 3,
 '3': 4,
 '째': 5,
 '2': 6,
 '정보': 7,
 '차원': 8,
 '데이터': 9,
 '자료': 10,
 '하': 11,
 '디': 12,
 '이': 13,
 'd': 14,
 '부탁': 15,
 '띄우': 16,
 '화': 17,
 '나타내': 18,
 '보이': 19,
 '그래프': 20,
 '표시': 21,
 '공유': 22,
 '넘어가': 23,
 '도표': 24,
 '시작': 25,
 '올리': 26,
 '7': 27,
 '8': 28,
 '과': 29,
 '3차원': 30,
 '6': 31,
 '3d': 32,
 '쓰리': 33,
 '범': 34,
 '로': 35,
 '말': 36,
 '다': 37,
 '1': 38,
 'ㄴ': 39,
 '2차원': 40,
 '를': 41,
 'ㄹ': 42,
 '반': 43,
 '투': 44,
 '4': 45,
 '켜': 46,
 '9': 47,
 '분': 48,
 '10': 49,
 '픽업': 50,
 '알리': 51,
 '자로': 52,
 '리': 53,
 '그': 54,
 '구': 55,
 '들': 56,
 '원': 57,
 '대': 58,
 '두': 59,
 '그러': 60,
 '박자': 61,
 '5': 62,
 '떼': 63,
 '그만': 64,
 '가': 65,
 '삼': 66,
 '종료': 67,
 '일': 68,
 '화면': 69,
 '띠': 70,
 '오': 71,
 '터': 72,
 '대리': 73,
 '육': 74,
 '음악': 75,
 '창원': 76,
 '꺼': 77,
 '음': 78,
 '소거': 79,
 '안': 80,
 '열': 81,
 '일곱': 82,
 '깨우': 83,
 '찍': 84,
 '공룡': 85,
 '팔': 86,
 '시간': 87,
 '여덟': 88,
 '깨': 89,
 '비우': 90,
 '뛰': 91,
 '자': 92,
 '이번': 93,
 '아': 94,
 '사람': 95,
 '여섯': 96,

In [ ]:
word_index['포인트']

260

In [ ]:
#시퀀스 최대 길이 구하기
maxlen_sequence=0
for review_number in range(len(sequences)):
  numberofwords=len(sequences[review_number])
  if (numberofwords) > (maxlen_sequence):
    maxlen_sequence=numberofwords
maxlen_sequence

14

In [ ]:
#시퀀스의 패딩 처리
padded_seqs=preprocessing.sequence.pad_sequences(sequences, maxlen=maxlen_sequence,
                                                 padding='post')

In [ ]:
#하이퍼 파라미터 설정
dropout_prob=0.2
embedding_size=128
epoch=3
word_size=len(word_index)+1
word_size

618

In [ ]:
type(labels[0])

int

In [ ]:
#데이터를 학습데이터, 검증데이터, 테스트 데이터로 분활
ds=tf.data.Dataset.from_tensor_slices((padded_seqs, labels))
ds=ds.shuffle(len(padded_seqs))

#train_size=int(len(padded_seqs)*0.9)
train_size=int(len(padded_seqs)*0.7)
val_size=int(len(padded_seqs)*0.2)
test_size=int(len(padded_seqs)*0.1)

train_ds=ds.take(train_size).batch(30)
val_ds=ds.skip(train_size).take(val_size).batch(30)
#test_ds=ds.skip(train_size).take(test_size).batch(30)
test_ds=ds.skip(train_size+val_size).take(test_size).batch(30)

In [ ]:
len(padded_seqs)

81539

In [ ]:
#Early Stoping 설정
early_stop = EarlyStopping(monitor='val_loss', mode='auto')

In [ ]:
from tensorflow.keras.layers import Bidirectional, LSTM, Dense, TimeDistributed

In [ ]:
#CNN모델 생성
input_layer =Input(shape=(maxlen_sequence,))
embedding_layer = Embedding(input_dim=word_size, output_dim=embedding_size, input_length=maxlen_sequence)(input_layer)
embedding_dropout=Dropout(rate=dropout_prob)(embedding_layer)

conv1=Conv1D(
    filters=128,
    kernel_size=2,
    padding='valid',
    activation=tf.nn.relu)(embedding_dropout)
lstm1=LSTM(64,recurrent_dropout=0.5,return_sequences=True)(conv1)
#lstm1=Bidirectional(LSTM(100,dropout=0.2,recurrent_dropout=0.2,return_sequences=True))(conv1)
pool1=GlobalMaxPool1D()(conv1)

conv2=Conv1D(
    filters=128,
    kernel_size=3,
    padding='valid',
    activation=tf.nn.relu)(embedding_dropout)
lstm2=LSTM(64,recurrent_dropout=0.5,return_sequences=True)(conv2)
#lstm2=Bidirectional(LSTM(100,dropout=0.2,recurrent_dropout=0.2,return_sequences=True))(conv2)
pool2=GlobalMaxPool1D()(conv2)

conv3=Conv1D(
    filters=128, 
    kernel_size=4,
    padding='valid',
    activation=tf.nn.relu)(embedding_dropout)
lstm3=LSTM(64,recurrent_dropout=0.5,return_sequences=True)(conv3)
#lstm3=Bidirectional(LSTM(100,dropout=0.2,recurrent_dropout=0.2,return_sequences=True))(conv3)
pool3=GlobalMaxPool1D()(conv3)

concat=concatenate([pool1,pool2,pool3])
hidden=Dense(128,activation=tf.nn.relu)(concat)
dropout_hidden=Dropout(rate=dropout_prob)(hidden)
logits=Dense(30, name='logits')(dropout_hidden)
predictions=Dense(30,activation='softmax')(logits)
model=Model(inputs=input_layer, outputs=predictions)
model.compile(loss='sparse_categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

In [ ]:
model.fit(train_ds, validation_data=val_ds, epochs=5, verbose=1)

In [ ]:
loss, accuracy=model.evaluate(test_ds, verbose=1)
print('Accuracy: %f' % (accuracy))
print('lossL %f'%(loss))

In [ ]:
#model.fit(train_ds, validation_data=val_ds, epochs=5, verbose=1)
model.fit(train_ds, epochs=3, verbose=1)

Epoch 1/3
2447/2447 [==============================] - 41s 17ms/step - loss: 0.2554 - accuracy: 0.9255
Epoch 2/3
2447/2447 [==============================] - 39s 16ms/step - loss: 0.0169 - accuracy: 0.9952
Epoch 3/3
2447/2447 [==============================] - 39s 16ms/step - loss: 0.0118 - accuracy: 0.9962


In [ ]:
loss, accuracy=model.evaluate(test_ds, verbose=1)
print('Accuracy: %f' % (accuracy))
print('lossL %f'%(loss))

272/272 [==============================] - 2s 5ms/step - loss: 0.0042 - accuracy: 0.9982
Accuracy: 0.998160
lossL 0.004224


In [ ]:
model.save('Contest_Cnn_Lstm_model11_CnnLstm_Capston2.h5')

In [ ]:
#최적의 하이퍼파라미터 값을 구하기 위한 튜닝
def build_model(hp):
  
  #embedding_size=128
  word_size=618
  maxlen_sequence=14
  #hp_units
  #튜닝할 하이퍼 파라미터 값들
  hp_embedding_size=hp.Choice('embeding_size',values=[64,128,256,512])
  hp_n_filters = hp.Int('n_filters', min_value=128, max_value=512, step=30)
  hp_dropout_prob=hp.Choice('dropout_units',values=[0.25 ,0.5, 0.75])
  hp_filter_size=hp.Choice('filter_size',values=[2,3,4])
  hp_recurrent_dropout_porb=hp.Choice('recurrent_dropout_units',values=[0.25,0.5,0.75])

  #입력계층
  input_layer =Input(shape=(maxlen_sequence,))
  #임베딩 계층
  embedding_layer = Embedding(input_dim=word_size, output_dim=hp_embedding_size, input_length=maxlen_sequence)(input_layer)
  dropout_emb=Dropout(rate=hp_dropout_prob)(embedding_layer)

  #1D CNN을 통한 특징 추출
  conv1=Conv1D(
    filters=hp_n_filters,
    kernel_size=hp_filter_size,
    padding='valid',
    activation=tf.nn.relu)(dropout_emb)
  #양방향 LSTM 결합(CNN_BiLstm)
  lstm1=LSTM(64,recurrent_dropout=hp_recurrent_dropout_porb,return_sequences=True)(conv1)
  #lstm1=Bidirectional(100,recurrent_dropout=hp_recurrent_dropout_porb,return_sequences=True)(conv1)
  #맥스풀링 연산 수행
  pool_1=GlobalMaxPool1D()(lstm1)

  #1D CNN을 통한 특징 추출
  conv2=Conv1D(
    filters=hp_n_filters, 
    kernel_size=hp_filter_size+1,
    padding='valid',
    activation=tf.nn.relu)(dropout_emb)
  #양방향 LSTM 결합(CNN_BiLstm)
  lstm2=LSTM(64,recurrent_dropout=hp_recurrent_dropout_porb,return_sequences=True)(conv2)
  #lstm2=Bidirectional(100,recurrent_dropout=hp_recurrent_dropout_porb,return_sequences=True)(conv2)
  #맥스풀링 연산 수행
  pool_2=GlobalMaxPool1D()(lstm2)

  #1D CNN을 통한 특징 추출
  conv3=Conv1D(
    filters=hp_n_filters,  
    kernel_size=hp_filter_size+2,
    padding='valid',
    activation=tf.nn.relu)(dropout_emb)
  #LSTM 결합(CNN_BiLstm)
  lstm3=LSTM(64,recurrent_dropout=hp_recurrent_dropout_porb,return_sequences=True)(conv3)
  #lstm3=Bidirectional(100,recurrent_dropout=hp_recurrent_dropout_porb,return_sequences=True)(conv3)
  #맥스풀링 연산 수행
  pool_3=GlobalMaxPool1D()(lstm3)
 
  #CNN_BiLstm을 병렬로 결합
  concat=concatenate([pool_1,pool_2,pool_3])

  #은닉층 생성
  hidden =Dense(hp_n_filters, activation=tf.nn.relu)(concat)
  dropout_hidden=Dropout(rate=dropout_prob)(hidden)
  #점수값(logits=score) 출력
  logits=Dense(30, name='logits')(dropout_hidden)
  #점수를 토대로 라벨(정답값예측)
  predictions=Dense(30, activation=tf.nn.softmax)(logits)

  model=Model(inputs=input_layer, outputs=predictions)
  model.compile(optimizer='adam',loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])
  return model

In [ ]:
#importing random search
from kerastuner import RandomSearch
#랜덤 서치 객체 생성
tuner = RandomSearch(build_model,
                    objective='val_accuracy',
                    max_trials = 5,
                     overwrite=True
                    )
# 최적의 모델 선택
tuner.search(train_ds,epochs=5,validation_data=val_ds)

In [ ]:
model=tuner.get_best_models(num_models=1)[0]
#summary of best model
model.summary()

In [ ]:
loss, accuracy=model.evaluate(test_ds, verbose=1)
print('Accuracy: %f' % (accuracy))
print('lossL %f'%(loss))

272/272 [==============================] - 9s 28ms/step - loss: 0.0065 - accuracy: 0.9978
Accuracy: 0.997792
lossL 0.006458


In [ ]:
model.save('Contest_Cnn_Lstm_model3_1.h5')

In [ ]:
import tensorflow as tf
from tensorflow.keras.models import Model, load_model
from tensorflow.keras import preprocessing
from konlpy.tag import Komoran

class IntentModel:
    def __init__(self, model_name):
        self.labels ={0: '0', 1: '1', 2: '2', 3: '3', 4: '4', 5: '5', 6: '6', 7: '7', 8: '8', 9: '9', 10: '10', 11: '11', 12: '12', 13: '13', 14: '14', 15: '15', 16: '16', 17: '17', 18: '18', 19: '19', 20: '20', 21: '21', 22: '22', 23: '23', 24: '24', 25: '25', 26: '26', 27: '27', 28: '28', 29: '29'}
        self.model = load_model(model_name)


    def predict_class(self, query):
        query=query.lower()
        komoran = Komoran()
        MAX_SEQ_LEN = 14
        input_text = komoran.pos(query)
        print(input_text)
        exclusion_tags=['EC','VX', 'XSV','SS','SF','SP','EF','JKO']
        word_list=[]
        for i in input_text:
            if i[1] not in exclusion_tags:
                word_list.append(i[0])
        print(word_list)
        word_index_dict = {}
        with open('ContestData_word_index11_Capston2_01.txt','r', encoding = 'UTF-8') as f:
        #with open('ContestData_word_index11.txt','r', encoding = 'UTF-8') as f:
        #with open('ContestData_word_index3.txt','r', encoding = 'UTF-8') as f:
            #'ContestData_word_index11_Capston2_01.txt'
            while True:
                line = f.readline()
                try:
                    word_index_dict[eval(line.split(':')[0].strip(' '))] = int(line.split(':')[1].rstrip(',\n'))
                except:
                    pass
                if not line : break
        #print('word_index_dictionary')
        #print(word_index_dict)
        w2i=[]
        for word in word_list:
            try:
                w2i.append(word_index_dict[word])
            except KeyError:
                pass
            
        sequences=[w2i]
        padded_sen=preprocessing.sequence.pad_sequences(sequences, maxlen=MAX_SEQ_LEN, padding='post')
        predict = self.model.predict(padded_sen)
        
        real_IDO=np.array(['포인트'])
        word_list=np.array(word_list)
        
        if (True in (np.in1d(word_list, real_IDO))):
          predict[0,0:22]=0
          predict[0,24:]=0

        predict_prob=np.max(predict[0])
        predict_class = tf.math.argmax(predict, axis = 1)

        return predict,predict_prob, predict_class.numpy()[0]

In [ ]:
intent=IntentModel('Contest_Cnn_Lstm_model11_CnnLstm_Capston2.h5')
query='오인트 꺼 줘!' 
predict,predict_prob,predict_class=intent.predict_class(query)
print('확률: {}'.format(predict_prob))
print('예측 라벨 값(23):{}'.format(predict_class))

[('오', 'NR'), ('인트', 'NNP'), ('꺼', 'NNB'), ('주', 'VX'), ('어', 'EF'), ('!', 'SF')]
['오', '인트', '꺼']
1/1 [==============================] - 0s 132ms/step
확률: 0.9999884366989136
예측 라벨 값(23):23


In [ ]:
intent=IntentModel('Contest_Cnn_Lstm_model3.h5')

query='비지엠 틀어줘' 
predict,predict_prob,predict_class=intent.predict_class(query)
print(predict)
print(type(predict))
print(predict_prob)
print(predict_class)

[('비', 'XPN'), ('지엠', 'NNP'), ('틀', 'VV'), ('어', 'EC'), ('주', 'VX'), ('어', 'EC')]
['비', '지엠', '틀']
word_index_dictionary
{'번': 2, '시각': 3, '3': 4, '째': 5, '2': 6, '정보': 7, '차원': 8, '데이터': 9, '자료': 10, '하': 11, '디': 12, '이': 13, 'd': 14, '부탁': 15, '띄우': 16, '화': 17, '나타내': 18, '보이': 19, '그래프': 20, '표시': 21, '공유': 22, '넘어가': 23, '도표': 24, '시작': 25, '올리': 26, '7': 27, '8': 28, '과': 29, '3차원': 30, '6': 31, '3d': 32, '쓰리': 33, '범': 34, '로': 35, '말': 36, '다': 37, '1': 38, 'ㄴ': 39, '2차원': 40, '를': 41, 'ㄹ': 42, '반': 43, '투': 44, '4': 45, '켜': 46, '9': 47, '분': 48, '10': 49, '픽업': 50, '알리': 51, '자로': 52, '리': 53, '그': 54, '구': 55, '들': 56, '원': 57, '대': 58, '두': 59, '그러': 60, '박자': 61, '5': 62, '떼': 63, '그만': 64, '가': 65, '삼': 66, '종료': 67, '일': 68, '화면': 69, '띠': 70, '오': 71, '터': 72, '대리': 73, '육': 74, '음악': 75, '창원': 76, '꺼': 77, '음': 78, '소거': 79, '안': 80, '열': 81, '일곱': 82, '깨우': 83, '찍': 84, '공룡': 85, '팔': 86, '시간': 87, '여덟': 88, '깨': 89, '비우': 90, '뛰': 91, '자': 92, '이번': 93, '아': 94, '사람

In [ ]:
#ContestData_word_index11.txt
#/content/Contest_Cnn_Lstm_model11_CnnLstm.h5

0.0545004047